<div class = "row">
    <div class = "colums">
        <img src="..\trecslogo.png" align="left" alt="Drawing" width ="60"/>    
    </div>
    <div class = "colums">
        <img src="..\asrlogo.png" align="right" alt="Drawing" width ="175"/>
    </div>    
</div>

# SWAPTION

## Algemeen

### Import en instellingen

In [1]:
import pandas as pd
import numpy as np
import datetime
import math
from scipy.optimize import fsolve
from scipy.stats import norm
from bokeh.plotting import figure, output_file, ColumnDataSource, output_notebook
from bokeh.models import HoverTool, NumeralTickFormatter, FactorRange
from bokeh.io import show
output_notebook(hide_banner=True)

Links uitlijnen tabellen

In [2]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

Bij printen van een dataframe wordt slechts een beperkt aantal rijen getoond.

In [3]:
pd.set_option('display.max_rows', 10)

## Initialisatie parameters

In [4]:
# data
datePricing = '20220331' # in dit notebook veronderstel ik dat de actuele datum dezelfde is als de kwartaal datum
datePricing = datetime.datetime.strptime(datePricing, '%Y%m%d')

# naam van de basis curve
curveNameBasis = 'FairValue'

# Bachelier model (instead of H&W)
Bachelier = True

# Haug formula for cash sattled swaptions assumes that 6 month compounded swap rate used as the discounting rate
m = 2

## Importeer en bewerk data

### Rentecurves per kwartaal en actueel
In de TRT is een begin gemaakt om te rekenen met de dual curve methodiek, dus op basis van de zero swap curve en de zero EONIA curve. Dit moet in TRT nog verder worden uitgewerkt en is nog niet operationeel. Dit notebook is daarom vooralsnog obv single curve methodiek (net zoals TRT).

In [5]:
df_curves = pd.read_excel(r"curvebestanden/curvesAssets.xlsx", decimal = '.')

pd.options.display.float_format = '{:,.8f}'.format
df_curves
#df_curves.to_excel('output.xlsx')

,Jaar,Currency,FairValue,swap.cra.zero.va.down,swap.cra.zero.va.up,SII_basis,SII_Yield_Curve_down,SII_Yield_Curve_up,SII_basis.EQUITY_TYPE_1,SII_basis.EQUITY_TYPE_2,...,swap.cra.eur-stylized-1.zero.va.sw345.down345,swap.cra.eur-stylized-1.zero.va.sw345.up345,swap.cra.dnb,swap.cra.dnb.up,swap.cra.dnb.down,swap.cra.zero.va-ratio50.sw270,swap.cra.zero.va-ratio50.sw270.down270,swap.cra.zero.va-ratio50.sw270.up270,swap.cra.zero.va-ratio50.sw270_Currency_Up,swap.cra.zero.va-ratio50.sw270_Currency_Down
0,1,EUR,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869,...,-0.00080776,0.00919224,-0.00080869,0.00919131,-0.00080869,-0.00080869,-0.00080869,0.00919131,-0.00080869,-0.00080869
1,2,EUR,0.00537754,0.00253036,0.01537754,0.00537754,0.00254137,0.01537754,0.00537754,0.00537754,...,0.00254794,0.01538412,0.00537754,0.01537754,0.00253036,0.00537754,0.00254137,0.01537754,0.00537754,0.00537754
2,3,EUR,0.00806788,0.00411486,0.01806788,0.00806788,0.00412935,0.01806788,0.00806788,0.00806788,...,0.00413130,0.01806983,0.00806788,0.01806788,0.00411486,0.00806788,0.00412935,0.01806788,0.00806788,0.00806788
3,4,EUR,0.00926595,0.00510382,0.01926595,0.00926595,0.00511846,0.01926595,0.00926595,0.00926595,...,0.00511910,0.01926659,0.00926595,0.01926595,0.00510382,0.00926595,0.00511846,0.01926595,0.00926595,0.00926595
4,5,EUR,0.00990248,0.00577112,0.01990248,0.00990248,0.00578518,0.01990248,0.00990248,0.00990248,...,0.00578534,0.01990264,0.00990248,0.01990248,0.00577112,0.00990248,0.00578518,0.01990248,0.00990248,0.00990248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00162101,0.01723252,0.00723252,0.00723252,...,0.00160880,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00273378,0.01723252,0.00723252,0.00723252
96,97,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00160777,0.01723252,0.00723252,0.00723252,...,0.00159568,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00272451,0.01723252,0.00723252,0.00723252
97,98,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00159479,0.01723252,0.00723252,0.00723252,...,0.00158284,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00271543,0.01723252,0.00723252,0.00723252
98,99,EUR,0.00723252,0.00541209,0.01723252,0.00723252,0.00158208,0.01723252,0.00723252,0.00723252,...,0.00157025,0.01723288,0.00723252,0.01723252,0.00541209,0.00723252,0.00270653,0.01723252,0.00723252,0.00723252


In [6]:
# curveNames = list(df_curves.columns)[2:]
curveNames = ['FairValue']
curveNames

['FairValue']

### IMW bestand

CIC codes swaptions:

- B6 = Call options granting its owner the right but not the obligation to enter into a long position in an underlying swap, i.e., enter into a swap where the owner pays the fixed leg and receive the floating leg
- C6 = Put options granting its owner the right but not the obligation to enter into a short position in an underlying swap, i.e., enter into a swap in which the owner will receive the fixed leg, and pay the floating leg

In [7]:
col_list = ['Reporting Date', 'Cic Id Ll', 'Market Value EUR LL', 'BalNomVal LT', 'Strike Price Laagste Level', 'Maturity Call LL', 'Maturity LL', 'Security Id Ll', 'Security Type Ll', 'Volatility dim']
df_swaption = pd.read_csv(r"imwbestand/2022M03 Adjusted IMW Weekupdate 202205131221.csv", usecols = col_list, sep = ";", decimal = '.', encoding= 'unicode_escape', low_memory=False)

In [8]:
df_swaption = df_swaption.loc[df_swaption['Security Type Ll'] == 'SWAPTION']
df_swaption = df_swaption.reset_index(drop=True)

In [9]:
df_swaption['dateIMW'] = pd.to_datetime(df_swaption['Reporting Date'], format='%Y%m%d')
df_swaption['dateSwapExpiry'] = pd.to_datetime(df_swaption['Maturity LL'], format='%Y%m%d')
df_swaption['dateSwaptionExpiry'] = pd.to_datetime(df_swaption['Maturity Call LL'], format='%Y%m%d')
df_swaption = df_swaption.drop(['Reporting Date', 'Security Type Ll', 'Maturity LL', 'Maturity Call LL'], axis = 1)

In [10]:
df_swaption.rename(columns = { \
                              'Cic Id Ll':'cicCode', \
                              'Security Id Ll':'securityId', \
                              'Market Value EUR LL':'marketValueIMW', \
                              'BalNomVal LT':'nominalValue', \
                              'Volatility dim':'volatilityPercentageIMW', \
                              'Strike Price Laagste Level':'strikePercentage' \
                             }, inplace = True)
pd.options.display.float_format = '{:,.2f}'.format
df_swaption

,cicCode,securityId,marketValueIMW,nominalValue,volatilityPercentageIMW,strikePercentage,dateIMW,dateSwapExpiry,dateSwaptionExpiry
0,XLB6,3350972,"2,332,319.63","50,000,000.00",0.58,2.75,2022-03-31,2068-06-03,2038-06-01
1,XLC6,3350574,"965,615.72","2,000,000.00",1.43,4.00,2022-03-31,2042-09-21,2022-09-19
2,XLC6,3350415,"1,454,048.72","3,000,000.00",1.54,4.00,2022-03-31,2042-07-28,2022-07-26
3,XLB6,3350971,"3,485,393.65","50,000,000.00",0.56,2.10,2022-03-31,2068-06-03,2038-06-01
4,XLB6,3350980,"1,790,950.35","35,000,000.00",0.57,2.60,2022-03-31,2068-07-29,2038-07-27
...,...,...,...,...,...,...,...,...,...
48,XLB6,3350756,"86,639.40","38,000,000.00",1.26,4.00,2022-03-31,2038-07-07,2023-07-05
49,XLB6,3350982,"2,406,877.36","50,000,000.00",0.58,2.70,2022-03-31,2068-08-04,2038-08-02
50,XLB6,3350983,"2,719,139.91","50,000,000.00",0.57,2.50,2022-03-31,2068-08-05,2038-08-03
51,XLB6,3350967,"5,318,994.74","125,000,000.00",0.59,2.90,2022-03-31,2068-05-26,2038-05-24


## Documentatie
In dit document staat de methode voor het berekenen van de swaptions beschreven. 

## Functie voor pricing swaptions

In [11]:
def getPriceModel(curve, swaptionType, cashSettled, tenor, expiry, volatility, strike):

#     print('swaptionType', swaptionType)
#     print('cashSettled', cashSettled)
#     print('tenor', round(tenor))
#     print('expiry', expiry)
#     print('volatility', volatility)
#     print('strike', strike)
     
    # make discounts from interest rate curve
    discounts = [1]
    for t in range(1, 100):
        discounts.append(1*(1+curve[t-1])**(-t))
    
    # make maturities for swap fixed cashflows (assumption is 6 months payment)
    maturitiesSwap = [] 
    for t in range(0, round(tenor)*2+1):
        maturitiesSwap.append(expiry+t*0.5)
    
    # make discounts related to swap fixed cashflows via linear interpolation
    maturities = range(0, 100)
    discountsSwap = np.interp(maturitiesSwap, maturities, discounts)
#     print('discountsSwap', discountsSwap[0])
    
    # calculate the forward rate with maturity equal to tenor and starting at expiry date
    forwardRate = (discountsSwap[0]/discountsSwap[-1])**(1/round(tenor))-1
#     print('forwardRate', forwardRate)
    
    if Bachelier:
        d = (forwardRate-strike)/(volatility*math.sqrt(expiry))
        z = 1
        if swaptionType == 'receiver':
            z = -1
        price = volatility*math.sqrt(expiry)*norm.pdf(d)+z*(forwardRate-strike)*norm.cdf(z*d)
        if cashSettled:
            factor = discountsSwap[0]*((1-1/(1+forwardRate/m)**(m*round(tenor)))/forwardRate)
            priceModel = price*factor
        else:
            sumDiscount = 0
            for t in range(1, len(discountsSwap)):
                sumDiscount += discountsSwap[t]/m
            priceModel = price*sumDiscount
    else: # Hull & White
        raise ValueError('Hull & White model is not yet implemented!')

    return priceModel

In [12]:
def findImpliedVolatilityModel(impliedVolatilityModel, *data):
    priceIMW = data[0]
    priceModel = getPriceModel(data[6], data[1], data[2], data[3], data[4], impliedVolatilityModel, data[5])
#     print(priceModel, priceIMW, impliedVolatilityModel)
    return priceModel-priceIMW

## Class voor swaptions

In [13]:
class Swaption:

    # Initialiseren van class argumenten obv meegegeven data
    # Bij het aanmaken van een nieuw object worden deze meteen geïnitialiseerd

    def __init__(self, dateIMW, cicCode, securityId, marketValueIMW, nominalValue, volatilityPercentageIMW, strikePercentage, dateSwapExpiry, dateSwaptionExpiry):
        self.dateIMW = dateIMW
        self.cicCode = cicCode
        self.securityId = securityId
        self.marketValueIMW = marketValueIMW
        self.nominalValue = nominalValue
        self.volatilityIMW = volatilityPercentageIMW/100
        self.strike = strikePercentage/100
        self.dateSwapExpiry = dateSwapExpiry
        self.dateSwaptionExpiry = dateSwaptionExpiry
        self.cashSettled = True # (voorlopige) aanname in TRT is dat alle swaptions cash settled zijn
        self.getSwaptionType()
        self.getTenor()
        self.getYearsToExpirySwaptionFromDateIMW()
        self.getPriceIMW()

    # Berekenen van class argumenten alleen obv geïnitialiseerde class argumenten
    # Bij het aanmaken van een nieuw object worden deze automatisch aangemaakt.

    def getSwaptionType(self):
        if self.cicCode[2:] == 'B6':
            self.swaptionType = 'payer'
        elif self.cicCode[2:] == 'C6':
            self.swaptionType = 'receiver'
        else:
            raise ValueError('Swaption type could not be found!')

    def getTenor(self):
        self.tenor = (self.dateSwapExpiry - self.dateSwaptionExpiry).days / 365

    def getYearsToExpirySwaptionFromDateIMW(self):
        self.yearsToExpirySwaptionFromDateIMW = (self.dateSwaptionExpiry - self.dateIMW).days / 365

    def getPriceIMW(self):
        self.priceIMW = self.marketValueIMW / self.nominalValue

    # Berekenen van class argumenten obv geïnitialiseerde en berekende class argumenten, en obv extra input die niet tot de class behoort
    # Bij het aanmaken van een nieuw object worden deze niet automatisch aangemaakt. 
    # De argumenten worden pas aangemaakt zodra de betreffende functie in de code wordt aangeroepen.
    
    def getMarketValueModel(self, curve):
        self.marketValueModel = getPriceModel(curve, self.swaptionType, self.cashSettled, self.tenor, self.getYearsToExpirySwaptionFromDatePricing(self.dateIMW), self.volatilityIMW, self.strike)*self.nominalValue
    
    def getImpliedVolatilityModel(self, curve):
        data = (self.priceIMW, self.swaptionType, self.cashSettled, self.tenor, self.getYearsToExpirySwaptionFromDatePricing(self.dateIMW), self.strike, tuple(curve))
        self.impliedVolatilityModel = fsolve(findImpliedVolatilityModel, self.volatilityIMW, data)[0]

    # Class argumenten worden niet aangemaakt, maar berekeningen zijn wel gebaseerd op de bestaande class argumenten, en obv extra input die niet tot de class behoort
    # De berekeningen worden dus niet in de class opgeslagen, dus moeten eventueel elders worden ondergebracht (bv dataframe) voor verder gebruik
    # Dat is in dit geval overigens niet aan de orde, i.e. de expiry wordt slechts eenmalig in de code aangemaakt en is verder niet nodig. 
    # Daarom is het ook niet nodig de berekening als argument op te slaan
    # In principe hoeft onderstaande method niet in de class te staan, maar zou ook als externe functie kunnen worden gedaan.
    # Reden om de method toch in de class te zetten is dat:
            # (ook) gebruik gemaakt wordt van class argumenten
            # de expiry echt een eigenschap is van de class
            # de code kort en overzichtelijk in de class kan worden opgenomen

    def getYearsToExpirySwaptionFromDatePricing(self, datePricing):
        yearsToExpirySwaptionFromDatePricing = (self.dateSwaptionExpiry - datePricing).days / 365
        return yearsToExpirySwaptionFromDatePricing

    # NB. De functie getPriceModel zou in principe ook als onderdeel van de class kunnen worden opgenomen.
    # Net als bij de expiry zou de berekening niet als argument van de class worden opgeslagen.
    # Ditmaal niet omdat de berekening slechts 1x nodig is, maar omdat de berekening voor heel veel curves moet worden gedaan.
    # Het is niet handig al deze berekeningen op te slaan als argumenten van de class, i.e. het is beter om daarvoor een dataframe te gebruiken
    # Reden om de functie niet in de class te zetten is dat:
            # de code relatief omvangrijk is en de class te veel zou vervuilen

## Aanmaken van alle swaption objecten en argumenten

### Aanmaken van alle swaption objecten en de initiële argumenten

In [14]:
swaptions = []
for index, row in df_swaption.iterrows():
    swaption = Swaption(row['dateIMW'], row['cicCode'], row['securityId'], row['marketValueIMW'], row['nominalValue'], row['volatilityPercentageIMW'], row['strikePercentage'], row['dateSwapExpiry'], row['dateSwaptionExpiry'])
    swaptions.append(swaption)

### Berekenen van het argument 'market value model' voor alle swaption objecten

In [15]:
curveBasis = df_curves[curveNameBasis].to_list()

In [16]:
for i in range(len(swaptions)):
    swaptions[i].getMarketValueModel(curveBasis)

### Berekenen van het argument 'implied volatility' voor alle swaption objecten

In [17]:
for i in range(len(swaptions)):
    swaptions[i].getImpliedVolatilityModel(curveBasis)

C:\Users\odries\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


## Berekenen van alle swaption waarden voor alle curves

In [18]:
columnNames = ['securityId'] + curveNames
df_swaptionMarketValuesModel = pd.DataFrame(columns = columnNames)
for i in range(len(swaptions)):
    df_swaptionMarketValuesModel.loc[i, 'securityId'] =  swaptions[i].securityId
    for curveName in curveNames:
        curve = df_curves[curveName].to_list()
        df_swaptionMarketValuesModel.loc[i, curveName] = getPriceModel(curve, swaptions[i].swaptionType, swaptions[i].cashSettled, swaptions[i].tenor, swaptions[i].getYearsToExpirySwaptionFromDatePricing(datePricing), swaptions[i].volatilityIMW, swaptions[i].strike) * swaptions[i].nominalValue \
                                                            + (swaptions[i].marketValueIMW - swaptions[i].marketValueModel)

In [19]:
df_swaptionMarketValuesModel

,securityId,FairValue
0,3350972,"2,332,319.63"
1,3350574,"965,615.72"
2,3350415,"1,454,048.72"
3,3350971,"3,485,393.65"
4,3350980,"1,790,950.35"
...,...,...
48,3350756,"86,639.40"
49,3350982,"2,406,877.36"
50,3350983,"2,719,139.91"
51,3350967,"5,318,994.74"


In [21]:
# df_swaptionMarketValuesModel.to_excel('output.xlsx')

## Output

## Verificatie